<a href="https://colab.research.google.com/github/theresaskruzna/riiid_knowledge_tracing/blob/main/01_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
# Riiid Knowledge Tracing Project
# Initial setup and data exploration

# This installs the Kaggle API package. The -q flag makes the installation "quiet" (less verbose output).
# This only needs to be run once per Colab session
!pip install -q kaggle

# This imports the files module from Google Colab and opens a file upload dialog where you select your Kaggle API credentials file (kaggle.json) from your local computer.
# Upload your kaggle.json file (from your Kaggle account settings)
from google.colab import files

import kagglehub
import os
import pickle

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import layers, models

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


# Data pre-processing

## Load original data

In [2]:
files.upload()  # Upload your kaggle.json here

!mkdir -p ~/.kaggle # Creates a .kaggle directory in your home folder (~) if it doesn't already exist. The -p flag ensures no error is thrown if the directory already exists.
!cp kaggle.json ~/.kaggle/ # Copies the uploaded kaggle.json file to the .kaggle directory where the Kaggle API expects to find it.
!chmod 600 ~/.kaggle/kaggle.json # Sets permissions on the kaggle.json file to "read/write for owner only" (600). This is required by the Kaggle API for security reasons.

# Download the dataset
!kaggle competitions download -c riiid-test-answer-prediction # Downloads all files for the Riiid Test Answer Prediction competition. The -c flag specifies which competition.
!unzip -q riiid-test-answer-prediction.zip -d riiid_data # Extracts the downloaded zip file into a directory called "riiid_data". The -q flag makes the unzip operation quiet (less verbose).

Saving kaggle.json to kaggle.json
 99% 1.27G/1.29G [00:10<00:00, 135MB/s]
100% 1.29G/1.29G [00:10<00:00, 134MB/s]


In [3]:
# This command will list all files and directories within the "riiid_data" directory in a detailed format.
!ls -la riiid_data/

total 5710060
drwxr-xr-x 3 root root       4096 Mar 15 16:33 .
drwxr-xr-x 1 root root       4096 Mar 15 16:33 ..
-rw-r--r-- 1 root root        971 Oct 14  2020 example_sample_submission.csv
-rw-r--r-- 1 root root       6137 Oct 14  2020 example_test.csv
-rw-r--r-- 1 root root       9703 Oct 14  2020 lectures.csv
-rw-r--r-- 1 root root     296161 Oct 14  2020 questions.csv
drwxr-xr-x 2 root root       4096 Mar 15 16:33 riiideducation
-rw-r--r-- 1 root root 5846760913 Oct 14  2020 train.csv


In [4]:
# This command lists detailed information about the specific file kaggle.json in the .kaggle directory in your home folder.
!ls -la ~/.kaggle/kaggle.json

-rw------- 1 root root 70 Mar 15 16:33 /root/.kaggle/kaggle.json


Loading questions.csv and lectures.csv into the notebook.

In [7]:
# Load the dataset with memory optimisations

# Define column data types to reduce memory usage
dtype_mapping_questions = {
    "question_id": "int16",
    "bundle_id": "int16",
    "correct_answer": "int8",
    "part": "int8",
    "tags": "object"
}

dtype_mapping_lectures = {
    "lecture_id": "int16",
    "part": "int8",
    "tag": "int16",
    "type_of": "object",
}

In [10]:
%%time

questions_df = pd.read_csv('riiid_data/questions.csv', dtype=dtype_mapping_questions)

CPU times: user 10.2 ms, sys: 901 µs, total: 11.1 ms
Wall time: 14.3 ms


In [11]:
%%time

lectures_df = pd.read_csv('riiid_data/lectures.csv', dtype=dtype_mapping_lectures)

CPU times: user 2.89 ms, sys: 0 ns, total: 2.89 ms
Wall time: 2.79 ms


## Load pickle dataset

The original train.csv data set is too large (5.85GB) for processing in collab, thus, a more suitable data set that has been compressed and converted from .csv to .pkl.gzip will be used for the time series forecasting purposes.
This approach downloads the whole dataset but I will look into ways to change it to dowloading just the single file I need.

In [12]:
# Download latest version
path = kagglehub.dataset_download("rohanrao/riiid-train-data-multiple-formats")

print("Path to dataset files:", path)

os.listdir(path)

100%|██████████| 4.17G/4.17G [00:40<00:00, 112MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rohanrao/riiid-train-data-multiple-formats/versions/3


['riiid_train.parquet',
 'riiid_train.feather',
 'riiid_train.pkl.gzip',
 'riiid_train.h5',
 'riiid_train.jay']

In [39]:
# Load the dataset with memory optimisations

# Define column data types to reduce memory usage
dtype_mapping_train = {
    "timestamp": "int64",
    "user_id": "int32",
    "content_id": "int16",
    "content_type_id": "boolean",
    "task_container_id": "int16",
    "answered_correctly": "int8",
    "prior_question_elapsed_time": "float32",
    "prior_question_had_explanation": "boolean"
}

# Define which colums to use
usecols_train=[1, 2, 3, 4, 5, 7, 8, 9],

In [40]:
%%time

# Load the dataset
train_df = pd.read_pickle(f"{path}/riiid_train.pkl.gzip")

CPU times: user 1.94 s, sys: 2.92 s, total: 4.86 s
Wall time: 27.1 s


Drop row_id and user_answer columns

In [41]:
# Check mememory usage before optimisation
def memory_usage(df):
    return f"{df.memory_usage(deep=True).sum() / 1024 ** 2:.2f} MB"

print(f"Dataset shape: {train_df.shape}")
print(f"Memory usage before optimisation: {memory_usage(train_df)}")

Dataset shape: (101230332, 10)
Memory usage before optimisation: 6463.74 MB


In [ ]:
# Drop the row_id and user_answer columns to save memory
train_df.drop(['row_id', 'user_answer'], axis=1, inplace=True) # axis=1 for columns, axis=0 for rows, inplace=True  modifies original df

# Check memory usage after dropping columns
print(f"Memory usage after dropping columns: {memory_usage(train_df)}")

Define column data types

In [31]:
# Cast columns to the desired data types
for column, data_type in dtype_mapping_train.items():
    train_df[column] = train_df[column].astype(data_type)  # Direct assignment

# Check memory usage after datatype conversions
print(f"Memory usage after datatype conversions: {memory_usage(train_df)}")

Memory usage after datatype conversions: 2413.52 MB


# EDA

## Feature exploration

### train

The 'timestamp' variable has values as high as 87 billion miliseconds which tells me nothing so I decided to convert it to days to get a better perspective on what that means for the dataset.

In [32]:
max_timestamp = train_df['timestamp'].max()
max_days = max_timestamp / (1000 * 60 * 60 * 24)  # Convert milliseconds to days
print(f"Maximum timestamp in days: {max_days}")

Maximum timestamp in days: 24.85513431712963


So thats about 2 and 3/4 of a year between interactions for the particular user so will have to explore if that is quite unique in the dataset and could be considered an outlier or if the time data really spans that much in general.

How many unique users in the dataset?

In [ ]:
num_unique_users = train_df['user_id'].nunique()
print(f"Number of unique users: {num_unique_users}")

Number of unique users: 393656


The 'content_id' is basically an 'question_id' form question.csv or 'lecture_id' from lecture.csv datasets and then the 'content_type_id' defines which one of these is it in the train.pkl.gzip dataset.

It's boolean where False means if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

In [ ]:
train_df.content_type_id.value_counts()

,count
content_type_id,
False,99271300
True,1959032


In [ ]:
num_unique_content = train_df['content_id'].nunique()
print(f"Number of unique content: {num_unique_content}")

# Calculate and print unique questions and lectures in one line
print(f"Unique questions: {train_df[train_df.content_type_id == False].content_id.nunique()}\nUnique lectures: {train_df[train_df.content_type_id == True].content_id.nunique()}")

Number of unique content: 13782
Unique questions: 13523
Unique lectures: 415


It seems that users are given questions in containers and then seeing explanations for them and potentially answers which then may hekp with answering the next batch of questions.

In [ ]:
unique_task_container = train_df['task_container_id'].nunique()
print(f"Number of unique task containers: {unique_task_container}")

Number of unique task containers: 10000


The variable 'user_answer' refers to what choice they picked out of multiple answer questions, numbers 0-3 refer to 4 choices, and number -1 means they engaged with a lecture instead.

In [ ]:
train_df.user_answer.value_counts()

,count
user_answer,
0,28186489
1,26990007
3,26084784
2,18010020
-1,1959032


The count for -1 (lectures) checks with the count in 'content_type_id'.

The variable 'answered_correctly' tells us if users got the question right(1) or wrong(0), or if they engaged with a lecture instead(-1). This is also the target variable for this analysis.

In [ ]:
answered_correctly_counts = train_df.answered_correctly.value_counts()

NameError: name 'train_df' is not defined

The maxiumum elapsed time since last question asked could also potentially have some outliers where people don't finish the container or leave the device withiut answering the question.

In [ ]:
max_elapsed_time = train_df['prior_question_elapsed_time'].max()
max_minutes = max_elapsed_time / 60000  # Convert milliseconds to minutes
print(f"Maximum elapsed time: {max_minutes}")

Maximum elapsed time: 5.0


There probably is a cut off at 5 minutes per question.

The last variable in the dataset is probably going to be quite imporatant to compare to predicting if the answers to questions were correct. It states in the description that in the beginning the questions asked were onboarding Qs so there wouldnt be feedback but as a student would progress on the platform they would be given some.

In [ ]:
train_df.prior_question_had_explanation.value_counts()


,count
prior_question_had_explanation,
True,89685560
False,11152266


In [ ]:
percentages = train_df.prior_question_had_explanation.value_counts(normalize=True) * 100
percentages

,proportion
prior_question_had_explanation,
True,88.940394
False,11.059606


### questions

question_id - How many unique questions in the dataset?

In [ ]:
num_unique_questions = questions_df['question_id'].nunique()
print(f"Number of unique questions: {num_unique_questions}")

Number of unique users: 393656


bundle_id - How many unique question bundles in the dataset?

In [ ]:
num_unique_bundles = questions_df['bundle_id'].nunique()
print(f"Number of unique question bundles: {num_unique_bundles}")

correct_answer

In [ ]:
questions_df.correct_answer.value_counts()

part - the relevant section of the TOEIC test.

https://www.iibc-global.org/english/toeic/test/lr/about/format.html

The Test of English for International Communication (TOEIC) is an international standardized test of English language proficiency for non-native speakers.

In [ ]:
questions_df.part.value_counts()

tags

In [ ]:
num_unique_tags = questions_df['tag'].nunique()
print(f"Number of unique tags: {num_unique_tags}")

### lectures

lecture_id - How many unique lectures in the dataset?

In [ ]:
num_unique_lectures = lectures_df['lecture_id'].nunique()
print(f"Number of unique lectures: {num_unique_lectures}")

Number of unique users: 393656


part - TOEIC test

https://www.iibc-global.org/english/toeic/test/lr/about/format.html

The Test of English for International Communication (TOEIC) is an international standardized test of English language proficiency for non-native speakers.

In [ ]:
lectures_df.part.value_counts()

tag

In [ ]:
num_unique_tags = lectures_df['tag'].nunique()
print(f"Number of unique tags: {num_unique_tags}")

type_of

In [ ]:
lectures_df.type_of.value_counts()

## Visualisations

### Functions

KDE plot

In [ ]:
def plot_kde(data, column, hue=None, title=None):
    """
    Creates a simple KDE plot for a specified column.

    Parameters:
    - data: Pandas DataFrame
    - column: Column name to plot
    - hue: Optional categorical column for grouping
    - title: Optional plot title
    """
    plt.figure(figsize=(8, 5)) # Create a 'canvas' to draw the plot onto
    # plt.figure(figsize=figsize) - alternate code if parameter given in function call
    sns.kdeplot(data=data, x=column, hue=hue, fill=True) # create KDE plot
    # df, column, split into categories if provided, fill area under line with colour

    # Custom labels and title
    plt.xlabel(column) # horizontal line label
    # plt.xlabel(xlabel if xlabel else column) - alternate code to add label
    plt.ylabel("Density") # vertical line label
    if title:
        plt.title(title) # add title to plot if provided in function call

    plt.show() # display plot

    # --- Added code for descriptive statistics ---
    # Assume 'column' is numerical by default
    categorical = False

    # Check if the column is of object (string) type, which often represents categorical data
    if data[column].dtype == 'object':
        categorical = True

    if categorical:
        values = data[column].value_counts().values
    else:
        values = data[column].values

    print('Mean: {:,}'.format(np.mean(values)))
    print('Median: {:,}'.format(np.median(values)))
    print('Max: {:,}'.format(np.max(values)))
    # --- End of added code ---

# Example usage:
# plot_kde(df, 'column_name', hue='category_column', title='KDE Plot')

# additional parameters:
# bw_adjust=1, color=None, xlabel=None, ylabel="Density", figsize=(8, 5)
 """
    Parameters:
    - bw_adjust: Bandwidth adjustment factor (default=1).
    - color: Optional color for the KDE curve.
    - xlabel: Optional x-axis label.
    - ylabel: Y-axis label (default="Density").
    - figsize: Tuple defining the figure size (default=(8,5)).
    """

Bar plot

In [ ]:
def plot_bar(data, x, y=None, title=None):
    """
    Creates a customizable bar plot.

    Parameters:
    - data: Pandas DataFrame containing the data.
    - x: Column name for the x-axis categories.
    - y: Optional column name for the y-axis values (if None, counts are used).
    - hue: Optional column for grouping (creates grouped bars).
    - color: Optional color for the bars.
    - title: Optional title for the plot.
    - xlabel: Optional x-axis label (default uses x column name).
    - ylabel: Optional y-axis label (default uses y column name or "Count").
    - figsize: Tuple defining the figure size (default=(8,5)).
    """
    plt.figure(figsize=(8, 5)) # Create a 'canvas' to draw the plot onto
    # plt.figure(figsize=figsize) - alternate code if parameter given in function call

    # Create either a value plot or count plot
    # If y is provided, plot values; otherwise plot counts
    if y:
        sns.barplot(data=data, x=x, y=y, hue=hue, color=color)
    else:
        sns.countplot(data=data, x=x, hue=hue, color=color)

    # Custom labels and title
    plt.xlabel(x) # horizontal line label
    # plt.xlabel(xlabel if xlabel else x) - alternate code to add label
    plt.ylabel(y if y else "Count") # vertical line label
    if title:
        plt.title(title) # add title to plot if provided in function call

    # Rotate x-axis labels if there are many categories to prevent label overlap
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout() # adjusts the plot padding to ensure all labels fit within the figure without being cut off

    plt.show() # display plot

# Example usage:
# plot_bar(df, x='category_column')                       # Simple count plot
# plot_bar(df, x='category_column', y='value_column')     # Value-based bar plot
# plot_bar(df, x='category_column', hue='group_column')   # Grouped bar plot

# additional parameters:
# hue=None, color=None, xlabel=None, ylabel=None, figsize=(8, 5)
    """
    Parameters:
    - hue: Optional column for grouping (creates grouped bars).
    - color: Optional color for the bars.
    - xlabel: Optional x-axis label (default uses x column name).
    - ylabel: Optional y-axis label (default uses y column name or "Count").
    - figsize: Tuple defining the figure size (default=(8,5)).
    """

Histogram

In [ ]:
def plot_histogram(data, column, bins=None, hue=None, title=None):
    """
    Creates a simple histogram for a specified column.

    Parameters:
    - data: Pandas DataFrame
    - column: Column name to plot
    - bins: Optional number of bins (none/default uses seaborn's auto bin selection)
    - hue: Optional categorical column for grouping
    - title: Optional plot title
    """
    plt.figure(figsize=(8, 5)) # Create a 'canvas' to draw the plot onto
    # plt.figure(figsize=figsize) - alternate code if parameter given in function call
    sns.histplot(data=data, x=column, bins=bins, hue=hue, kde=True) # create histogram
    # df, column, number of bins, split into categories if provided, line on top of the histogram bars

    # Custom labels and title
    plt.xlabel(column) # horizontal line label
    # plt.xlabel(xlabel if xlabel else column) - alternate code to add label
    plt.ylabel("Count") # vertical line label
    if title:
        plt.title(title) # add title to plot if provided in function call

    plt.show() # display plot

# Example usage:
# plot_histogram(df, 'column_name', bins=20, hue='category_column', title='Histogram')

# additional parameters:
# color='blue', xlabel=None, ylabel="Count", figsize=(10, 6), kde=False)
"""
    Parameters:
    - color: Optional color for the histogram bars.
    - xlabel: Optional x-axis label (default uses column name).
    - ylabel: Y-axis label (default="Count").
    - figsize: Tuple defining the figure size (default=(8,5)).
    - kde: Boolean to include KDE curve overlay (default=True).
    """

### timestamp and prior_question_elapsed_time

In [ ]:
plot_kde(train_df, 'timestamp', hue=None, title=None)

In [ ]:
plot_kde(train_df, 'prior_question_elapsed_time', hue=None, title=None)

Alternate code

In [ ]:
numerical_columns = ['timestamp', 'prior_question_elapsed_time']

for column in numerical_columns:
    plot_kde(train_df, column=column, hue=None, title=column + " distribution")

Histogram for timestamp? Most distribution is within the first 2 months of signing up to the platform - every user starts with 0 value for their first log - does the number of 0 values for timestamp check with number of users?

Check unique number count for user_id

In [ ]:
plot_histogram(train_df, 'timestamp', bins=20, hue=None, title=None)

### user_id, content_id and task_container_id

In [ ]:
plot_kde(train_df, 'user_id', hue=None, title=None)

In [ ]:
plot_kde(train_df, 'content_id', hue=None, title=None)

In [ ]:
plot_kde(train_df, 'task_container_id', hue=None, title=None)

Alternate code

In [ ]:
categorical_columns = ['user_id', 'content_id', 'task_container_id']

for column in categorical_columns:
    plot_kde(train_df, column=column, hue=None, title=column + " countplot distribution")

### content_type_id, answered_correctly and prior_question_had_explanation

In [ ]:
plot_bar(train_df, x='content_type_id', y=None, title=None)

In [ ]:
plot_bar(train_df, x='answered_correctly', y=None, title=None)

In [ ]:
plot_bar(train_df, x='prior_question_had_explanation', y=None, title=None)

Alternate code

In [ ]:
categorical_columns_few = ['content_type_id', 'answered_correctly', 'prior_question_had_explanation']

for column in categorical_column_few:
    plot_bar(train_df, x=column, y=None, title=column + " barplot")

### question_id, bundle_id and tags

In [ ]:
plot_kde(questions_df, 'question_id', hue=None, title=None)

In [ ]:
plot_kde(questions_df, 'bundle_id', hue=None, title=None)

In [ ]:
plot_kde(questions_df, 'tags', hue=None, title=None)

### correct_answer and part

In [ ]:
plot_bar(questions_df, x='correct_answer', y=None, title=None)

In [ ]:
plot_bar(questions_df, x='part', y=None, title=None)

### lecture_id and tags

In [ ]:
plot_kde(lectures_df, 'lecture_id', hue=None, title=None)

In [ ]:
plot_kde(lectures_df, 'tags', hue=None, title=None)

### part and type_of

In [ ]:
plot_bar(lectures_df, x='part', y=None, title=None)

In [ ]:
plot_bar(lectures_df, x='type_of', y=None, title=None)

## Correlation

### Functions

In [ ]:
def plot_heatmap(data, annot=True, cmap="viridis", title=None, xlabel=None, ylabel=None):
    """
    Creates a customizable heatmap.

    Parameters:
    - data: DataFrame or 2D array to visualize as heatmap.
    - annot: Whether to annotate cells with values (default=True).
    - cmap: Colormap name or object (default="viridis").
    - title: Optional title for the plot.
    - xlabel: Optional x-axis label.
    - ylabel: Optional y-axis label.
    """
    plt.figure(figsize=(8, 5)) # Create a 'canvas' to draw the plot onto
    # plt.figure(figsize=figsize) - alternate code if parameter given in function call

    # Create the heatmap
    heatmap = sns.heatmap(data, annot=annot, cmap=cmap)

    # Custom labels and title
    if xlabel:
        plt.xlabel(xlabel) # horizontal line label
    if ylabel:
        plt.ylabel(ylabel) # vertical line label
    if title:
        plt.title(title) # add title to plot if provided in function call

    plt.tight_layout() # adjusts the plot padding to ensure all labels fit within the figure without being cut off
    plt.show() # display plot

    return heatmap # return the heatmap object for further customization if needed

# Example usage:
# plot_heatmap(df.corr())                             # Correlation heatmap
# plot_heatmap(pivot_table, cmap="YlOrRd", fmt=".0f") # Custom colored heatmap
# mask = np.triu(np.ones_like(df.corr(), dtype=bool)) # Upper triangle mask
# plot_heatmap(df.corr(), mask=mask, cmap="coolwarm") # Masked heatmap

    # additional parameters
    # center=None, robust=False, mask=None, figsize=(10, 8), vmin=None, vmax=None, fmt=".2f", linewidths=0, cbar=True
    """
    Parameters:
    - center: Value to center the colormap at (useful for diverging data).
    - robust: If True, compute colormap bounds from robust percentiles.
    - mask: Boolean array or DataFrame to mask cells (True = hidden).
    - figsize: Tuple defining the figure size (default=(10,8)).
    - vmin, vmax: Limits for the colormap.
    - fmt: String formatting for annotations.
    - linewidths: Width of lines between cells.
    - cbar: Whether to draw a colorbar.
    """

### Heat map

## Target variable = answered_correctly

First I would like to see if the number of appearences of 1 ID (1 pupil) corresponds to their overall performance (other words, if you assess 1 student multiple times, will they perform better?)

In [ ]:
# Count how many times the user answered correctly out of all available times
user_performance = train.groupby("user_id").agg({ 'row_id': ['count'], 'answered_correctly': ['sum'] }).reset_index()
user_performance.columns = ["user_id", "total_count", "correct_count"]
user_performance["performance"] = user_performance["correct_count"] / user_performance["total_count"]

# Create intervals for number of appearences
# between 0 and 1000, 1000 and 2500 and 2500+
def condition(x):
    if x <= 1000:
        return 0
    elif (x > 1000) & (x <= 2500):
        return 1
    else:
        return 2

user_performance["total_interval"] = user_performance["total_count"].applymap(condition)

📌Note: So yes, the average performance increases along with the number of times one student appears in the data.

In [ ]:
# Convert to numpy arrays (so we can plot)
x = cupy.asnumpy(user_performance["total_interval"].values)
y = cupy.asnumpy(user_performance["performance"].values)

# Plot
plt.figure(figsize = (18, 4))
sns.barplot(x = x, y = y, palette = custom_colors[2:])
plt.title("Performance over number of appearences", fontsize = 15)
plt.xticks([0, 1, 2], ['<1000', '1000-2500', '2500+']);

del user_performance

# Feature engineering

## Missing values

### train.pkl.gzip

In [ ]:
missing_values = train_df.isna().sum().to_frame('missing_values')
missing_values

,missing_values
row_id,0
timestamp,0
user_id,0
content_id,0
content_type_id,0
task_container_id,0
user_answer,0
answered_correctly,0
prior_question_elapsed_time,2351538
prior_question_had_explanation,392506


📌Note: The only 2 columns with missing data (explained in documentation - NULL values are present for the first question bundle)

In [ ]:
# Data Information
print("Rows: {:,}".format(len(train)), "\n" +
      "Columns: {}".format(len(train.columns)))

# Find Missing Data if any
total = len(train)

for column in train.columns:
    if train[column].isna().sum() != 0:
        print("{} has: {:,} ({:.2}%) missing values.".format(column, train[column].isna().sum(),
                                                             (train[column].isna().sum()/total)*100))


# Fill in missing values with "-1"
train["prior_question_elapsed_time"] = train["prior_question_elapsed_time"].fillna(-1)
train["prior_question_had_explanation"] = train["prior_question_had_explanation"].fillna(-1)

train.head()

Rows: 101,230,332

Columns: 10

prior_question_elapsed_time has: 2,351,538 (2.3%) missing values.

prior_question_had_explanation has: 392,506 (0.39%) missing values.

### questions.csv

In [ ]:
missing_values = questions_df.isna().sum().to_frame('missing_values')
missing_values

,missing_values
question_id,0
bundle_id,0
correct_answer,0
part,0
tags,1


In [ ]:
missing_tag_row = questions_df[questions_df['tags'].isnull()]
missing_tag_row

,question_id,bundle_id,correct_answer,part,tags
10033,10033,10033,2,6,NaN


Only 1 missing value for the 'tags' variable - at this size of a dataset, the entire row can probably be deleted, however, as the question_id is related to the same variable in the main dataset, it's important to check what is the reprasentation of this particular question in that dataset.


In [ ]:
missing_question_id = missing_tag_row['question_id'].iloc[0]  # Extract the question_id
question_id_counts = train_df[train_df['content_id'] == missing_question_id].shape[0]

print(f"Occurrences of question_id {missing_question_id}: {question_id_counts}")

Occurrences of question_id 10033: 1


why are the two codes below not working?

In [ ]:
question_id_counts = train_df.query('content_id == 10033 and answered_correctly != -1').shape[0]

1

In [ ]:
row = train_df.loc[train_df['content_id'] == 10033]
print(row)

            row_id   timestamp     user_id  content_id  content_type_id  \
62750278  62750278  1286282597  1333688829       10033            False   

          task_container_id  user_answer  answered_correctly  \
62750278               1127            2                   1   

          prior_question_elapsed_time  prior_question_had_explanation  
62750278                      35000.0                            True  


The question_id 10033 is only found once across the whole train.pkl.gzip dataset with 101+ million rows so it's safe to say we can delete that row from question.csv.

In [ ]:
questions_df = questions_df.dropna(subset=['tags'])  # Drop rows with missing 'tags'

## Outliers

### Functions

In [ ]:
def plot_boxplot(data, x=None, y=None, hue=None, color=None, title=None, horizontal=False):
    """
    Creates a customizable box plot.

    Parameters:
    - data: Pandas DataFrame containing the data.
    - x: Column name for the x-axis (categorical).
    - y: Column name for the y-axis (numeric).
    - hue: Optional column for additional grouping.
    - color: Optional color for the boxes.
    - title: Optional title for the plot.
    - horizontal: Boolean to plot horizontally (default is vertical).
    """
    # Set orientation
    orient = 'h' if horizontal else 'v'

    plt.figure(figsize=(8, 5)) # Create a 'canvas' to draw the plot onto
    # plt.figure(figsize=figsize) - alternate code if parameter given in function call
    sns.boxplot(data=data, x=x, y=y, hue=hue, orient=orient) # Create the boxplot

    # Custom labels and title
    if horizontal:
        plt.xlabel(y)  # In horizontal orientation, y values are on x-axis
        plt.ylabel(x)  # In horizontal orientation, x values are on y-axis
    else:
        plt.xlabel(x)
        plt.ylabel(y)
        # In vertical orientation, x values are on x-axis, y values are on y-axis
    if title:
        plt.title(title) # add title to plot if provided in function call

    # Handle label rotation for vertical plots with many categories
    if orient == 'v' and x is not None:
        plt.xticks(rotation=45, ha='right')

    plt.tight_layout() # adjusts the plot padding to ensure all labels fit within the figure without being cut off
    plt.show() # display plot

# Example usage:
# plot_boxplot(df, x='category', y='value')                  # Basic vertical boxplot
# plot_boxplot(df, x='value', y='category', orient='h')      # Horizontal boxplot
# plot_boxplot(df, x='category', y='value', hue='group')     # Grouped boxplot

    # additional parameters
    # xlabel=None, ylabel=None, figsize=(8, 5), orient='v', showfliers=True, palette=None
    """
    Parameters:
    - color: Optional color for the boxes.
    - xlabel: Optional x-axis label.
    - ylabel: Optional y-axis label.
    - figsize: Tuple defining the figure size (default=(8,5)).
    - orient: Orientation of the plot ('v' for vertical, 'h' for horizontal).
    - showfliers: Whether to show outlier points (default=True).
    - palette: Color palette for groups when hue is specified.
    """

### timestamp

📌Note: The outliers might strongly influence the future models. Hence, we need to carefully handle them. However, by trying to erase the outliers we can erase up to 10% of the data, which is valuable information for training our models.

Eg.: Below we find the upper boundry for the outliers for the feature timestamp. This value is equal to 26,138,936,852 milliseconds, which means 7,700 hours, which means about 320 days.

In [ ]:
# Total rows we started with
total = len(train)
feature = "timestamp"

# Compute Outliers
Q1 = cupy.percentile(train[feature].values, q = 25).item()
Q3 = cupy.percentile(train[feature].values, q = 75).item()
IQR = Q3 - Q1

# We'll look only at the upper interval outliers
outlier_boundry = Q3 + 1.5*IQR

print('Timestamp: around {:.2}% of the data would be erased.'.format(len(train[train[feature] >= outlier_boundry])/total * 100),
      "\n"+
      'The outlier boundry is {:,}, which means {:,.5} hrs, which means {:,.5} days.'.format(outlier_boundry, (outlier_boundry / 3.6e+6),
                                                                                       (outlier_boundry / 3.6e+6)/24))

gc.collect()

Timestamp: around 9.4% of the data would be erased.

The outlier boundry is 24,024,861,068.375, which means 6,673.6 hrs, which means 278.07 days.

### Is there a way to find out whats the engagement count among users?

📌Note: However, I would erase all pupils (user_id) that have less than 5 appearences in the data (no prediction can be made on these students)

In [ ]:
# Select ids to erase
ids_to_erase = train["user_id"].value_counts().reset_index()[train["user_id"].value_counts().reset_index()["user_id"] < 5]\
                                                                                                                ["index"].values

# Erase the ids
new_train = train[~train['user_id'].isin(ids_to_erase)]

print("We erased {} rows meaning {:.3}% of all data.".format(len(train)-len(new_train), (1 - len(new_train)/len(train))*100))
del ids_to_erase

We erased 599 rows meaning 0.000592% of all data.

# Merging datasets

Goal is to merge train, questions and lectures datasets for model building

question_id is a foreign key for the train/test content_id column, when the content type is question (0)

lecture_id is a foreign key for the train/test content_id column, when the content type is lecture (1)

# Extra code


In [ ]:
plt.figure(figsize=(8, 6))  # Adjust figure size if needed
sns.countplot(data=train_df, x='answered_correctly')
plt.title('Distribution of answered_correctly')  # Add a title for clarity
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels if they overlap
plt.show()

In [ ]:
# Create a DataFrame for plotting
plot_df = pd.DataFrame({'answered_correctly': answered_correctly_counts.index,
                        'count': answered_correctly_counts.values})

plt.figure(figsize=(8, 6))
sns.barplot(data=plot_df, x='answered_correctly', y='count')
plt.title('Distribution of answered_correctly (Full Dataset)')
plt.xticks(rotation=45, ha='right')
plt.show()